In [8]:
from datetime import datetime
import numpy as np
from numba import cuda
from tabulate import tabulate
import matplotlib.pyplot as plt

from numba.cuda.random import create_xoroshiro128p_states, xoroshiro128p_uniform_float32 
# внутри видеокарты создавать рандомные списки

In [9]:
TPB = 16
count_iter = 16


In [10]:
'''
  Подсчёт числа Пи на CPU
  Смысл:
    Используется метод Монтекарло, через квадрат и вписанной окружности
    Мы создаем случайные X,Y размером от 0 до 1.
    Если сумма квадратов X и Y меньше или равно 1, то она в досигаемости центра
    А если больше, то она лежит за пределеами круга.
    Создаём такое некое количесвто и умножаем на 4 (по кол-ву секторов)
    А затем делим на их количество, N - обшее кол-во точек
'''

def Calcul_on_CPU(count_iter, N):
    res = np.zeros(count_iter)
    for i in range(count_iter):
        x = np.random.uniform(size=N)
        y = np.random.uniform(size=N)
        z = x ** 2 + y ** 2 <= 1
        res[i] = 4.0 * sum(z) / N
    return res

In [11]:
@cuda.jit 
def pi_calcul(res, rng):
  i = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x
  #выделяем сетку в CUDA
  h = 0 #кол-во точек лежащих внутри круга
  if i < len(rng):
    x = xoroshiro128p_uniform_float32(rng, i)
    y = xoroshiro128p_uniform_float32(rng, i)
    # создаём в CUDA свои случайные X,Y 
    if x ** 2 + y ** 2 <= 1.0:
      h += 1
  cuda.atomic.add(res, 0, h)
  #res засовываем в h

In [12]:
def Calcul_on_GPU(N):

  rng_states = create_xoroshiro128p_states(N, seed=1)
  #создаём спец.переменную, которая будет помогать создавать
  #списки внутри CUDA, прописывается заранее на CPU, а затем передаётся на GPU
  
  #CUDA не может сама создать списки, но
  #но rng будет внутренней переменной в GPU и тогда сможет
  #то есть это спец.переменная позволяющая это сделать
  res = [0] # результирующая

  dev_res = cuda.to_device(res)#загружаем в GPU
  pi_calcul[N, TPB](dev_res, rng_states)
  tmp_ = dev_res.copy_to_host()# потом выводим на CPU
  pi_ = 4 * tmp_[0] / N
  return pi_

In [13]:
def main(iter_ , N):
  #Эксперименты на CPU и GPU
  for i in range(iter_):
    print("Номер итерации:" + str(i+1))
    start = datetime.now()
    cpu_pi = Calcul_on_CPU(count_iter, N).mean()
    print("Время выполения на CPU - "+ str(datetime.now()- start))
    print("Число Pi - " + str(cpu_pi))
    start = datetime.now()
    gpu_pi = Calcul_on_GPU(N)
    print("Время выполения на GPU - "+ str(datetime.now()- start))
    print("Число Pi - " + str(gpu_pi))
    print("")
    N = N + 10000
  return None

In [14]:


main(5, 10000)



Номер итерации:1
Время выполения на CPU - 0:00:00.379006
Число Pi - 3.1504
Время выполения на GPU - 0:00:00.199388
Число Pi - 3.1552

Номер итерации:2
Время выполения на CPU - 0:00:00.767045
Число Pi - 3.1431
Время выполения на GPU - 0:00:00.007479
Число Pi - 3.1578

Номер итерации:3
Время выполения на CPU - 0:00:01.113933
Число Pi - 3.1424666666666665
Время выполения на GPU - 0:00:00.011508
Число Pi - 3.1508

Номер итерации:4
Время выполения на CPU - 0:00:01.484555
Число Pi - 3.1427250000000004
Время выполения на GPU - 0:00:00.018254
Число Pi - 3.1471

Номер итерации:5
Время выполения на CPU - 0:00:01.850575
Число Pi - 3.14401
Время выполения на GPU - 0:00:00.016778
Число Pi - 3.14304

